In [1]:
from ipywidgets import Video
import cv2
import numpy as np
import copy
import time
from functools import reduce
import operator
import math

In [2]:
def clock_wise(coords):
    center = tuple(map(operator.truediv, reduce(lambda x, y: map(operator.add, x, y), coords), [len(coords)] * 2))
    a,b,c,d=sorted(coords, key=lambda coord: (-135 - math.degrees(math.atan2(*tuple(map(operator.sub, coord, center))[::-1]))) % 360, reverse=True)
    sorted_points=[d,a,b,c]
    return sorted_points

def findMaxContour(image):
    #przygotowywanie pod edge detection
    gaussian = cv2.medianBlur(image,5,cv2.BORDER_DEFAULT)

    edges = cv2.Canny(gaussian,50,200)
    contours, hierarchy = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(image, contours, -1, (0,255,0), 3)

    contour = max(contours, key = cv2.contourArea)
    return contours, contour

def getResultImage(image, show = True):
    width, height, depth = image.shape
    if show:
        cv2.imshow("frame", image)
        print("Ready")
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contours, contour = findMaxContour(image)
        
    cv2.drawContours(image, contour, -1, (0,255,0), 3)

    #wklejenie konturów na czarnym tło
    black = np.zeros((width, height), dtype = np.uint8)
    black_contour = cv2.drawContours(black, contours, -1, (255,255,255), 3)
    contours, contour = findMaxContour(black_contour)
    
    #jeszcze raz szukanie w konturach konturu
    black = np.zeros((width, height), dtype = np.uint8)
    black_contour = cv2.drawContours(black, contour, -1, (255,255,255), 3)


#     #using erode function on the input image to be eroded
    black_contour = cv2.GaussianBlur(black_contour,(5,5),cv2.BORDER_DEFAULT) #cv2.erode(graycontourImg,(7,7),iterations = 2)
    if show:
        cv2.imshow("black_contour",black_contour )
        
#     # wyszukiwanie kątów poprzez szukanie 4 najwiekszych feature to track
    corners = cv2.goodFeaturesToTrack(black_contour, 4, 0.01, 100)
    corners = np.int0(corners)

    points_1 = []
    
    contourImg = cv2.drawContours(image, contour, -1, (0,255,0), 3)
    #problem with dots
    a=50
    for c in corners:
        cv2.circle(black_contour, *c ,5, (a,0,0), cv2.FILLED) 
        points_1.append(*c)
        a+=50
        
    points_1=clock_wise(points_1)
    if show:
        print(points_1)
        cv2.imshow("black_contour with dots", black_contour)
    
    points_1 = np.float32(points_1)
    x, y = (round(width/5), round(height/5))
    cx = 0
    cy = 0
    lx, ly = (500, 300)
    points_2 = np.float32([[x+cx,y+cy], [x+lx+cx, y+cy], [x+lx+cx, y+ly+cy],[x+cx, y+ly+cy]]) #od lewego górnego rogu w prawo
    resultimage = cv2.getPerspectiveTransform(points_1, points_2)

    finalimage = cv2.warpPerspective(image, resultimage, (height, width))
    if show:
        cv2.imshow("Frame cards", finalimage) 
    if show:
        key = cv2.waitKey(0) 
        if key == 32: 
            cap.release()
        cv2.destroyAllWindows() 
        
    return resultimage, points_2 

def calibrateImage(image, matrix):
    return cv2.warpPerspective(image, matrix, (image.shape[1], image.shape[0]))

In [3]:
def boxIntoRectangle(points):
    a,b,c,d = points
    return [(int(a[0]),int(a[1])) , (int(c[0]),int(c[1]))]

In [4]:
filename = 'easy.mp4'
card = cv2.imread('karta_1.png', cv2.IMREAD_COLOR)
coin = cv2.imread('moneta.png', cv2.IMREAD_COLOR)

w_card = card.shape[1]
h_card = card.shape[0]

w_coin = coin.shape[1]
h_coin = coin.shape[0]

cap = cv2.VideoCapture(filename)
if not cap.isOpened():
    print("Cannot open video")
    exit()
    
count = 0
matrix = None

thres_dice = .55
thres_coin = .65
while True:    
    
#image reading
    ret, frame = cap.read()
    
    
    while ret == False:
        print("Can't receive frame. Retrying ...")
        cap.release()
        cap = cv2.VideoCapture(filename)                                                                              
        ret, frame = cap.read()
    
#calibration step
    image = copy.deepcopy(frame)
    
    if count ==0:
        matrix, points_2 = getResultImage(copy.copy(image), False)
    
    frame = calibrateImage(image, matrix)
    
    roi_dice = copy.deepcopy(frame)


# cards step
    if count%20== 0:
        
        result = cv2.matchTemplate(frame, card, cv2.TM_CCOEFF_NORMED)
        
        y_loc, x_loc = np.where(result>=thres_dice)
        
        #robimy tak bo się grupują tylko gyd są min.2
        rectangles = []
        for (x,y) in zip(x_loc, y_loc):
            rectangles.append([int(x),int(y),int(x+w_card), int(y+h_card)])
            rectangles.append([x,y,x+w_card, y+h_card])
        rectangles, weights = cv2.groupRectangles(rectangles,1,0.1) #w ile ma się połączyć, jak blisko siebie by być już osobno
        
        for (x,y,w,h) in rectangles:
            cv2.rectangle(frame, (x,y),(w,h), (255,0,255),3)
    
        trackers=[]
        for r in rectangles:
            x,y,w,h = r
            tracker = cv2.legacy.TrackerMOSSE_create()
            tracker.init(frame, [x,y,w-x,h-y] )
            trackers.append(tracker)
            
    bottom_card = [100000,100000]        
    for tracker in trackers:
        success, bbox = tracker.update(frame)
        if success:
            (x,y,w,h) = [int(x) for x in bbox]
            cv2.rectangle(frame,(x,y), ((x+w),(y+h)), (255,0,255),1) 
            
            cv2.rectangle(roi_dice, (x-10,y-10), ((x+w+10),(y+h+10)), (255,0,255), -1) 
            if x<bottom_card[0]:
                bottom_card[0] = x
            if y<bottom_card[1]:
                bottom_card[1] = y+h
                 
    
    cv2.rectangle(roi_dice, *boxIntoRectangle(points_2), (255,0,255),-1)
    
#dice step
    offset = [int(points_2[0][0]), bottom_card[1]]
    roi_dice = roi_dice[offset[1]:,offset[0]:]
    
    roi_dice = cv2.GaussianBlur(roi_dice, (7,7),0) 
    kernel = np.ones((7, 7), 'uint8')
    
    thres = cv2.adaptiveThreshold(cv2.cvtColor(roi_dice, cv2.COLOR_BGR2GRAY), 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)
    
    thres = cv2.morphologyEx(thres, cv2.MORPH_CLOSE,
                           kernel, iterations=2)
    thres = cv2.morphologyEx(thres, cv2.MORPH_ERODE,
                           kernel, iterations=1)
    cv2.imshow("thres",thres)
    
    if count%1==0:
        conours, _ = cv2.findContours(thres, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    matching_contours = 0
    
    
    for cnt in conours:
        #calculate area and remove small noise
        rect = cv2.minAreaRect(cnt) 
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        h, w = (rect[1][1],rect[1][0])
    
        if w>25 and h>25 and h*w<5000: #1600
            cv2.drawContours(frame,[np.add(box, offset)],0,(0,0,255),2) #czerwony box
            cv2.drawContours(roi_dice,[box],0,(0,0,255),2) #czerwony box
            matching_contours+=1
            
#coin detection 
    result = cv2.matchTemplate(frame[0:round(frame.shape[0]/2),0:round(frame.shape[1]/2)], coin, cv2.TM_CCOEFF_NORMED)
    cv2.imshow("roi",frame[0:round(frame.shape[0]/2),0:round(frame.shape[1]/2)])
    y_loc, x_loc = np.where(result>=thres_coin)

    #robimy tak bo się grupują tylko gyd są min.2
    rectangles = []
    for (x,y) in zip(x_loc, y_loc):
        rectangles.append([int(x),int(y),int(x+w_coin), int(y+h_coin)])
        rectangles.append([x,y,x+w_coin, y+h_coin])
    rectangles, weights = cv2.groupRectangles(rectangles,1,0.01) #w ile ma się połączyć, jak blisko siebie by być już osobno

    for (x,y,wi,ha) in rectangles:
        cv2.rectangle(frame, (x,y),(wi,ha), (0,255,255),3)
    cv2.putText(frame, 
        'coin: '+str(len(rectangles)), 
        (50, 50), cv2.FONT_HERSHEY_SIMPLEX,1,
        (0, 255, 0), 
        2, cv2.LINE_AA)
 
    cv2.imshow("roi_dice", roi_dice) 
    cv2.imshow("Frame cards", frame) 
    #cv2.imshow("Frame", image) 
    count += 1
    key = cv2.waitKey(30) 
    if key == 32: 
        cap.release()
        cv2.destroyAllWindows() 
        break 


Cannot open video
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying 

Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive 

Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive 

Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive 

Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive 

...

Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive 

Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive 

Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive 

Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive frame. Retrying ...
Can't receive 

KeyboardInterrupt: 

In [17]:
cv2.waitKey(0)
cv2.destroyAllWindows() 